<a href="https://colab.research.google.com/github/khanhquynh25/DemoGit/blob/master/x%C4%91mgti%E1%BB%81n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from google.colab import drive

# ================== 1. KẾT NỐI GOOGLE DRIVE ==================
drive.mount("/content/drive")

# Thư mục chứa dataset trong Drive
base_dir = "/content/drive/MyDrive/xacdinhmgt1"

# ================== 2. TIỀN XỬ LÝ DỮ LIỆU ==================
img_size = (128, 128)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.5  # 20% dữ liệu làm validation
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    subset="training"
)

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    subset="validation"
)

# ================== 3. XÂY DỰNG MÔ HÌNH ANN ==================
model = Sequential([
    Flatten(input_shape=(img_size[0], img_size[1], 3)),
    Dense(512, activation="relu"),
    Dropout(0.4),
    Dense(256, activation="relu"),
    Dropout(0.3),
    Dense(len(train_generator.class_indices), activation="softmax")
])

# ================== 4. COMPILE ==================
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# ================== 5. TRAIN ==================
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30
)

# ================== 6. LƯU MÔ HÌNH VÀO DRIVE ==================
save_path = "/content/drive/MyDrive/xacdinhmgt1/money_ann.h5"
model.save(save_path)
print(f"✅ Đã train xong và lưu mô hình ANN tại: {save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 11 images belonging to 9 classes.
Found 8 images belonging to 9 classes.
Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0909 - loss: 3.4734 - val_accuracy: 0.1250 - val_loss: 12.8968
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0909 - loss: 21.4994 - val_accuracy: 0.1250 - val_loss: 18.4340
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 642ms/step - accuracy: 0.2727 - loss: 16.5710 - val_accuracy: 0.0000e+00 - val_loss: 10.7912
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1818 - loss: 26.1486 - val_accuracy: 0.2500 - val_loss: 18.8111
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 598ms/step - accuracy: 0.4545 - loss: 16.4731 - val_accuracy: 0.2500 - val_loss: 21.8745
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 620ms/step - accuracy: 0.3636 - loss: 13.1243 - val_accuracy: 0.2500 - val_loss: 23.5245
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━

✅ Đã train xong và lưu mô hình ANN tại: /content/drive/MyDrive/xacdinhmgt1/money_ann.h5


In [ ]:
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array

# Hàm dự đoán
def predict_and_show(img):
    img_resized = img.resize(img_size)   # resize về (128,128)
    img_array = img_to_array(img_resized) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)
    class_idx = np.argmax(preds)
    confidence = np.max(preds)
    label = labels[class_idx]

    return f"💵 Đây là mệnh giá: {label} nghìn\n(Độ tin cậy: {confidence:.2f})"

# ================== CSS cho nền hồng + nút đỏ + chữ kết quả to/đậm ==================
custom_css = """
.gradio-container {background-color: #ffe6f0; font-family: Arial;}
h1, h2, h3 {color: #cc0052; font-weight: bold;}
label {color: #99003d; font-weight: bold;}
button {background-color: #ff1a1a !important; color: white !important;
        font-weight: bold; border-radius: 10px !important; padding: 10px 20px !important;}
.output-textbox textarea {font-size: 22px !important; font-weight: bold !important; color: #cc0000;}
"""

demo = gr.Interface(
    fn=predict_and_show,
    inputs=gr.Image(type="pil", label="📷 Upload ảnh tiền"),
    outputs=gr.Textbox(label="📌 Kết quả dự đoán"),
    title="💵 Nhận diện mệnh giá tiền",
    description="Upload ảnh tiền để dự đoán mệnh giá (theo đơn vị nghìn).",
    css=custom_css
)

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b958e443fbf5d57cc2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Using existing dataset file at: .gradio/flagged/dataset3.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
